In [1]:
import pandas as pd
import random
from itertools import permutations

In [116]:
# CONSTANTS
"""
extra emojis!!! (watch out for odd extra hidden characters, will add silly columns to your dataframe)
👺⚡🎯🛡️⭐🐍🔴🔵
🗡️⚔️🪓🛡️💥🔥🟥🟦🆘
🧌💀💩👾👹😈👿👽
⏸️☣️🦂🤢☠️💫✨➕👘👕🎽
🤺🥷🏼🧙‍♂️😵💤🛌🔮🐉☆🧪🕸🕸️⚠️❌🪖🧱🕷
⏸😵‍💫⛊🔰⛨🛡️🩸
"""

# rolls
red = '🟥'
blue = '🟦'
hit = '💥'
shield = '⛨'
lightning = '⚡'
SS = '🐉'
star = '☆'
goblin = '😈'

# effects
armour = '🧱'
KO = '🛌'
stun = '⏸'
poison = '🤢'
critical = '❌'
lethal = '💀'
slow = '🕸'
bleed = '🩸'

In [152]:
# check for junk inside of used emojis
emojis = [red, blue, hit, shield, lightning, SS, star, goblin, armour, KO, stun, poison, critical, lethal, slow, bleed]

s = ''.join([x for x in emojis])
set([x for x in s if x != '️'])

{'⏸',
 '☆',
 '⚡',
 '⛨',
 '❌',
 '🐉',
 '💀',
 '💥',
 '🕸',
 '😈',
 '🛌',
 '🟥',
 '🟦',
 '🤢',
 '🧱',
 '🩸'}

In [118]:
# some don't show up with the above method? use effects datastructure

effects = [
    (lightning+lightning, stun), 
    (lightning, hit),
    (lightning, slow),
    (lightning+lightning, hit+hit),
    (star, poison),
    (star, shield),
    (star, armour),
    (star, lethal),
    (star, critical),
    (star, red),
    (star, blue),
    (star, lightning),
    (star, SS),
    (star, KO),
    (star, goblin),
    (star, bleed)
]

s = ''.join(sorted([x[1] for x in effects]))
set([x for x in s if x])

{'⏸', '⚡', '⛨', '❌', '🐉', '💀', '💥', '🕸', '😈', '🛌', '🟥', '🟦', '🤢', '🧱', '🩸'}

In [153]:
class Red:
    POSSIBLE_RESULTS = [
        [SS],
        [hit, hit],
        [lightning, lightning],
        [hit, lightning],
        [hit, lightning],
        [hit, lightning],
        [hit, star],
        [hit, star],
        [hit],
        [goblin]
    ]
    
    def __init__(self):
        self.colour = red
    
    def roll(self):
        return [self.colour] + random.choice(self.POSSIBLE_RESULTS)

    
class Blue(Red):
    POSSIBLE_RESULTS = [
        [shield],
        [shield],
        [shield],
        [shield],
        [hit, lightning],
        [hit, lightning],
        [hit, star],
        [hit, star],
        [hit, star],
        [goblin]
    ]
    
    def __init__(self):
        self.colour = blue

    
class Dice:
    def __init__(self, n_red=0, n_blue=0):
        self.n_red = n_red
        self.n_blue = n_blue
        self._dice = [Red() for _ in range(n_red)] + [Blue() for _ in range(n_blue)]
        self.current_roll = None
    
    def roll(self):
        result = []
        for die in self._dice:
            die_result = die.roll()
            result.append(die_result)
        self.current_roll = result
        return self.current_roll
    
    def reroll(self, n_to_reroll):
        """zero indexed - 1st item in n_to_reroll=0"""
        reroll = self._dice[n_to_reroll].roll()
        self.current_roll[n_to_reroll] = reroll
        return self.current_roll
    
    
class Item:
    def __init__(self, dice=None, modifiers=None, effects=None):
        self.dice = dice
        self.modifiers = modifiers
        self.effects = effects
        self.current_roll = None
        self.current_totals = None
        
    def roll(self, focus=False):
        if self.dice:
            result = self.dice.roll()
        else:
            result = []
        for k,v in self.modifiers.items():
            modifier_list = ['modifiers']
            for _ in range(v):
                modifier_list.append(k)
        result.append(modifier_list)
        if focus:
            result.append(['focus', hit])
        self.current_roll = result    
        self.calc_totals()
        return self.current_roll
    
    def reroll(self, n_to_reroll):
        if self.current_roll[n_to_reroll][0] == 'modifiers':
            print('cannot reroll modifiers')
            return self.current_roll
        self.current_roll[n_to_reroll] = self.dice.reroll(n_to_reroll)[n_to_reroll]
        self.calc_totals()
        return self.current_roll
    
    def print_dice(self):
        if self.current_roll:
            for die in self.current_roll:
                print(die)
            print()
            
    def calc_totals(self):
        if self.current_roll:
            final_results = dict()
            for roll in self.current_roll:
                for result in roll:
                    if result != red and result != blue and result != 'modifiers':
                        final_results[result] = final_results.get(result, 0) + 1
        self.current_total = final_results

    def print_totals(self):
        for result in self.current_total.items():
            print(result)


class Weapon(Item):
    def print_dice(self):
        if self.current_roll:
            for die in self.current_roll:
                print(die)
            print()

    def print_totals(self):
        if self.current_roll:
            final_results = dict()
            for roll in self.current_roll:
                for result in roll:
                    if result != red and result != blue and result != 'modifiers':
                        final_results[result] = final_results.get(result, 0) + 1
        for result in final_results.items():
            if result[0] == hit:
                print(result)
        for result in final_results.items():  # second loop guarantees hits on top
            if result[0] == hit:
                continue
            if self.effects:
                # TODO: need a better data structure for this, could be non-unique
                #  consider adding this to an attribute, for ref in other methods
                all_relevant_results = {x for x in self.effects for x in x}
                if result[0] in all_relevant_results:
                    print(result)
                    
    def results(self):
        """check all effects and list all potential outcomes, incl no effects"""
        fx = all_permutations(self.effects)
        results = []
        for r in self.current_roll:
            results += [x for x in r[1:]]
        results = ''.join(sorted(results, reverse=True))
        columns = list(set(hit + ''.join(sorted([x[1] for x in self.effects]))))  # always show hits
        output = pd.DataFrame(columns=columns)

        for f in fx:
            current_result = results 
            if len(f) > 0:
                for cost, effect in f:
                    current_result = current_result.replace(cost, effect, 1)

            counts = {col: current_result.count(col) for col in output.columns}
            output = pd.concat([output, pd.DataFrame([counts])])

        output = output.drop_duplicates().sort_values(hit, ascending=False).reset_index(drop=True)
        return output
            
            
# helper functions
def all_permutations(lst):
    """convert a list into a generator for all permutations of that list"""
    for i in range(len(lst)+1):
        for p in permutations(lst, r=i):
            yield p

# simulation
def simulation(obj, n_trials=100):
    pass


In [154]:
# when Thorgren is using the spear
spear = Weapon(
    Dice(2, 2), 
    modifiers={hit: 1},
    effects=[(lightning, hit), (lightning, armour), (lightning+lightning, bleed)]
)

In [178]:
spear.roll(True)
spear.current_roll

[['🟥', '💥', '☆'],
 ['🟥', '💥', '☆'],
 ['🟦', '💥', '☆'],
 ['🟦', '💥', '⚡'],
 ['modifiers', '💥'],
 ['focus', '💥']]

In [179]:
spear.results()

,💥,🧱,🩸
0,7,0,0
1,6,0,0
2,6,1,0


In [194]:
spear.roll(focus=True)
for x in spear.current_roll:
    print(x)
display(spear.results())

['🟥', '💥', '☆']
['🟥', '💥', '💥']
['🟦', '💥', '☆']
['🟦', '💥', '⚡']
['modifiers', '💥']
['focus', '💥']


,💥,🧱,🩸
0,8,0,0
1,7,0,0
2,7,1,0


In [195]:
# write the results function

effects = [
    (lightning+lightning, stun), 
    (lightning, hit),
    (lightning, slow),
    (lightning+lightning, hit+hit),
    (star, poison)
]

current_roll = [
    ['🟥', '💥', '⚡'],
    ['🟥', '⚡', '⚡'],
    ['🟦', '💥', '☆'],
    ['🟦', '💥', '☆'],
    ['modifiers', '💥'],
    ['focus', '💥']
]

def results(effects, current_roll):
    """recursively check all effects and list all potential outcomes, incl no effects"""
    fx = all_permutations(effects)
    results = []
    for r in current_roll:
        results += [x for x in r[1:]]
    results = ''.join(sorted(results, reverse=True))
    columns = list(set(hit + ''.join(sorted([x[1] for x in effects]))))  # always show hits
    output = pd.DataFrame(columns=columns)

    for f in fx:
        if len(f) < 1:
            continue
        
        current_result = results
        for cost, effect in f:
            current_result = current_result.replace(cost, effect, 1)
        
        counts = {col: current_result.count(col) for col in output.columns}
        output = pd.concat([output, pd.DataFrame([counts])])
    
    output = output.drop_duplicates().sort_values(hit, ascending=False).reset_index(drop=True)
    return output
        
df = results(effects, current_roll)
df

,💥,🤢,⏸,🕸
0,8,0,0,0
1,8,1,0,0
2,7,0,0,1
3,7,1,0,0
4,7,1,0,1
5,7,0,0,0
6,6,0,1,0
7,6,0,0,1
8,6,0,0,0
9,6,1,0,0
